<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/Sentence_embedding_with_FastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')
import zipfile
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import  stopwords                        # 향후 전처리로 to, for 등을 제거할 것

In [ ]:
# PC 에서 train.txt(log file 이름)를 upload

from google.colab import files     # Colab에서 local PC내의 file 읽어 들이는 라이브러리
uploaded = files.upload()          # train.txt 파일을  선택

In [ ]:
pre_processed_text = []

f = open('train.txt', 'r', encoding='UTF8')          # 1,100 줄의 log 파일
lines = f.readlines()

for line in lines:
    sent_text = sent_tokenize(line)     #  log msg를 줄 단위의 sentence로 토큰화
    #print(sent_text)

    # 각 문장에 대해서 구두점, LF 등을 제거하고, 대문자를 소문자로 변환.
    for string in sent_text:
        tokens = re.sub(r"[^a-z]+", " ", string.lower())         # 입력 파일(train.txt)에 있는 숫자를 제거했음(3월 1일에)
        #tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
        pre_processed_text.append(tokens)

print("pre_processed_text=", pre_processed_text)

result=[pre_processed_text]    # 문장 단위로 유사도를 비교할 때 , result=[[['rsyslogd origin softwarersyslogd swversion ', 'systemd starting rotate log files',...]]

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
#result = [word_tokenize(sentence) for sentence in pre_processed_text]    # 단어의 유사도를 비교하거나... 또는 한 문장과 유사도가 비슷한 단어를 찾을 때, result=[[['rsyslogd', 'origin', 'softwarersyslogd swversion'], ['systemd starting rotate log files'],,...]]
print("result=", result)
print('총 샘플의 개수 : {}'.format(len(result)))


In [ ]:
# FastText로 훈련하기---- min_count를 2로 할때 model 크기가 약 44MB, 1로 바꾸어 학습한 경과는 93MB 임. 즉, 한번 등장하는 문장이 많다는 의미...

from gensim.models import FastText
model = FastText(result, size=200, window=5, min_count=1, workers=4, sg=1)   # default epoch = 5 임. skip_gram이 사용하는 Window size=5

model.save('fasttext_train_0301')

In [ ]:
#a= model['logrotateservice succeeded']   # model['abc'] 로 벡터를 구하는 것은 gensim에서 deprecated 되었음.

a= model.wv['logrotateservice succeeded']   # get the vector of the word or sentence
b= model.wv['logrotateservice']
c= model.wv['succeeded']
#print(a)
print("a_sum=", sum(a))
print("b_sum=", sum(b))
print("c_sum=", sum(c))


In [ ]:
a= model.wv['systemd started message day']   # get the vector of the word or sentence
b= model.wv['systemd']
c= model.wv['started']
d= model.wv['message']
e= model.wv['day']

print("a_sum=", sum(a))
print("b_sum=", sum(b))
print("c_sum=", sum(c))
print("d_sum=", sum(d))
print("e_sum=", sum(e))

z= -(0.33092026179656386 +0.09879262174945325 +0.01677472274604952 +0.0649924342869781)/4  # 평균
print(z)

In [ ]:
model.wv.most_similar('systemd started message  day', topn=5)

In [ ]:
model.wv.most_similar(negative=['systemd started message  day'], topn=5, )

In [ ]:
model.wv.most_similar('fail', topn=5)    # most_similar 명령은 deprecated 됨.  wv.most_similar로 통합됨. --> snetence를 학습했으므로 한 단어를 입력해도 단어가 아닌 유사한 문장이 출력됨.

In [ ]:
!apt-get install glove_python  # !pip도 build 오류 발생함, toy로 소개되어 있어서 ???

# 구글 Drive에 연결하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

구글 Drive에 FastText 학습 모델 저장하기

In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/fasttext_train_0301') # Colab 좌측 창에 임시 저장된 h5 파일을 다운로드 하여 저장한다.

# EOF

In [ ]:
# Glove로 훈련하기  --> import
from glove import Glove, Corpus
#from gensim.models import Glove, Corpus
corpus = Corpus()
corpus.fit(result, window=5)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.